1. Introduction
==

In the previous missions, we learned how the linear regression model estimates the relationship between the feature columns and the target column and how we can use that for making predictions. In this mission and the next, we'll discuss the 2 most common ways for finding the optimal parameter values for a linear regression model. Each combination of unique parameter values forms a unique linear regression model, and the process of finding these optimal values is known as **model fitting**. Both approaches to model fitting we'll explore aim to minimize the following function:

$$MSE=\frac{1}{n}\sum_{i=1}^n(\hat{y}_i−y_i)^2$$


This function is the mean squared error between the predicted labels made using a given model and the true labels. The problem of choosing a set of values that minimize or maximize another function is known as an [optimization problem](https://en.wikipedia.org/wiki/Mathematical_optimization).

To build intuition for the optimization process, let's start with a single parameter linear regression model:

$$\hat{y}=a_1x_1$$

Note that this is different from a simple linear regression model, which actually has two parameters: $x_0$ and $x_1$.

$$\hat{y}=a_1x_1+a_0$$

Let's use the **Gr Liv Area** column for the single parameter:

$$\hat{SalePrice}=a_1∗GrLivArea$$




In [2]:
import pandas as pd
data = pd.read_csv('AmesHousing.txt', delimiter="\t")
train = data[0:1460]
test = data[1460:]

target = 'SalePrice'

In [3]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [4]:
from bokeh.models import ColumnDataSource
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider, Paragraph
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#################### Rigth figure - MSE vs A1 #####################
# create all poins to initial plot for MSE and a1
all_a1 = np.arange(-400,600)
y_mse = np.zeros(1000)
for i in all_a1:
    y_mse[i] = sum((1/train['Gr Liv Area'].shape[0]) * (all_a1[i]*train['Gr Liv Area'] - train['SalePrice'])**2)
    
# create source
source_msea1 = ColumnDataSource(data=dict(x=all_a1, y=y_mse))

# create a figure
p2 = figure(x_axis_label = 'a1',
           y_axis_label = 'MSE',
           plot_width = 400,
           plot_height = 400)

# initial plot
p2.circle(source=source_msea1, x="x", y="y")

# create source to a single red circle
source_point = ColumnDataSource({'x': [0], 'y': [y_mse[400]]})
p2.circle('x', 'y',source=source_point,size=10,fill_color='red')

#################### Left figure - Scatter plot: Gr Liv Area vs SalePrice #####################
# create scatter dataset
source = ColumnDataSource(train)

# initial linear regression model
x = np.linspace(0,3600,1000)
y = 250*x + 0

# create linear regression dataset
source_reg = ColumnDataSource(data=dict(x=x, y=y))

# create a figure
p = figure(x_axis_label = 'Gr Liv Area',
           y_axis_label = 'SalePrice',
           plot_width = 400,
           plot_height = 400)

# scartter plot
p.circle(source=source, x="Gr Liv Area", y="SalePrice")

# regression plot
p.line('x', 'y', source=source_reg, line_width=3, line_alpha=0.6, color='red')


# interactive part
update_curve = CustomJS(args=dict(source=source_reg,
                                 source2=source,
                                 source3=source_point), code="""
    var data = source.data;
    var data2 = source2.data;
    var data3 = source3.data;
    x = data['x']
    y = data['y']
    x2 = data3['x']
    y2 = data3['y']
    x_real = data2['Gr Liv Area']
    y_real = data2['SalePrice']
    a_0 = slider_a0.value;
    a_1 = slider_a1.value;
    rss = 0
    for (i = 0; i < x.length; i++) {
       y[i] = x[i]*a_1 + a_0
    }
    for (i = 0; i < x_real.length; i++){
        y_hat = a_1*x_real[i] + a_0
        rss = rss + Math.pow(y_real[i] - y_hat,2)
    }
    x2[0] = a_1
    y2[0] = rss/x_real.length
    paragraph.text = "RSS = " + rss.toString()
    source.change.emit();
    source3.change.emit();
""")


slider_a1 = Slider(start=-400.0, end=600, value=0, step=1, 
                  title="A1", callback=update_curve)
update_curve.args["slider_a1"] = slider_a1

slider_a0 = Slider(start=-300000, end=300000, value=0, step=1000, 
                  title="A0",callback=update_curve)
update_curve.args["slider_a0"] = slider_a0

paragraph = Paragraph(text="RSS = ---")
update_curve.args["paragraph"] = paragraph


show(column(paragraph,row(p,p2),slider_a1, slider_a0))


show(p)


2. Single Variable Gradient Descent
==

In the last screen's widget, we observed how the optimization function follows a curve with a minimum value. **This should remind you of our exploration of relative minimum values from calculus**. If you recall, we computed the critical points by calculating the curve's derivative, setting it equal to 0, and finding the x value at this point. Unfortunately, this approach won't work when we have multiple parameter values because minimizing one parameter value may increase another parameter's value. In addition, while we can plot the MSE curve when we only have a single parameter we're trying to find and visually select the value that minimizes the MSE, this approach won't work when we have multiple parameter value because we can't visualize past 3 dimensions.

In this mission, we'll explore an iterative technique for solving this problem, known as **gradient descent**. The [gradient descent algorithm](https://en.wikipedia.org/wiki/Gradient_descent) works by iteratively trying different parameter values until the model with the lowest mean squared error is found. Gradient descent is a commonly used optimization technique for other models as well, like neural networks, which we'll explore later in this track.

Here's an overview of the gradient descent algorithm for a single parameter linear regression model:

- select initial values for the parameter: $a_1$
- repeat until convergence (usually implemented with a max number of iterations):
    - calculate the error (MSE) of model that uses current parameter value: $MSE(a_1)=\frac{1}{n}\sum_{i=1}^{n}(\hat{y}^{(i)}−y^{(i)})^2$
    - calculate the derivative of the error (MSE) at the current parameter value: $\frac{d}{da_1}MSE(a_1)$
    - update the parameter value by subtracting the derivative times a constant ($\alpha$, called the learning rate): $a_1:=a_1−\alpha \frac{d}{da_1}MSE(a_1)$
    
    
In the last step of the algorithm, you'll notice we used we used := to indicate that the value on the right is assigned to the variable on the left. While in Python, we've used to the equals operator (=) for assignment, we've used it in math (=) to signify equality. For example, a = 1 in Python assigns the value 1 to the variable a. In math, a=1 asserts that a is equal to 1. In mathematical papers, sometimes $\leftarrow$ is also used to signify assignment:

$a_1\leftarrow a_1 − \alpha \frac{d}{da_1}MSE(a_1)$

Selecting an appropriate initial parameter and learning rate will reduce the number of iterations required to converge, and is part of hyperparameter optimization. We won't dive into those techniques in this course and will instead focus on how the algorithm works. In the next screen, we'll unpack how to calculate the derivative of the error function at each iteration of the algorithm.

3. Derivative Of The Cost Function
==

In mathematical optimization, a function that we optimize through minimization is known as a **cost function** or sometime as the [loss function](https://en.wikipedia.org/wiki/Loss_function). Because we're trying to fit a single parameter model, we can replace with y^(i) with a1x1(i) in the cost function:

$$MSE(a_1)=\frac{1}{n}\sum_{i=1}^{n}(a_1x_1^{(i)}−y^{(i)})^2$$

In this screen, we'll apply calculus properties to simplify this derivative to something we can compute. **We encourage you to follow along using pencil and paper, and see if you can apply the properties we mention at each step to obtain the same result we did**. Note that while you'll probably never have to implement gradient descent yourself (as most packages have high performance implementations), understanding the math will help make it easier for you to debug when you run into issues.

$$\frac{d}{da_1}MSE(a_1)=\frac{d}{da_1}\frac{1}{n}\sum_{i=1}^{n}(a_1x_1^{(i)}−y^{(i)})^2$$


By applying the [linearity of differentiation](https://en.wikipedia.org/wiki/Linearity_of_differentiation) property from calculus, we can bring the derivative term inside the summation:

$$\frac{d}{da_1}MSE(a_1)=\frac{1}{n}\sum_{i=1}^{n}\frac{d}{da_1}(a_1x_1^{(i)}−y^{(i)})^2$$

We can apply both the power rule and the chain rule to simplify this. You can read more about the chain rule [here](https://en.wikipedia.org/wiki/Chain_rule) or observe how both are applied together [here](https://www.khanacademy.org/math/calculus-home/taking-derivatives-calc/chain-rule-calc/v/differentiating-powers-of-functions):

$$\frac{d}{da_1}MSE(a_1)=\frac{1}{n}\sum_{i=1}^{n}2(a_1x_1^{(i)}−y^{(i)})\frac{d}{da_1}(a_1x_1^{(i)}−y^{(i)})$$

Because we're differentiating $a_1x_1^{(i)}−y^{(i)}$ with respect to $a_1$, we treat $y^{(i)}$ and $x_1^{(i)}$ as constants. $\frac{d}{da_1}(a_1x_1^{(i)}−y^{(i)})$ then simplifies to just $x_1^{(i)}$:

$$\frac{d}{da_1}MSE(a_1)=\frac{2}{n}\sum_{i=1}^{n}x_1^{(i)}(a_1x_1^{(i)}−y^{(i)})$$

For every iteration of gradient descent:

- this derivative is computed using the current $a_1$ value
- the derivative is multiplied by the learning rate ($\alpha$): $\alpha \frac{d}{da_1}MSE(a_1)$
- the result is subtracted from the current parameter value and assigned as the new parameter value: $a1:=a_1−\alpha \frac{d}{da_1}MSE(a_1)$

Here's what this would look like in code if we ran gradient descent for **10** iterations:

>```python
a1_list = [1000]
alpha = 10
for x in range(0, 10):
    a1 = a1_list[x]
    deriv = derivative(a1, xi_list, yi_list)
    a1_new = a1 - alpha*deriv
    a1_list.append(a1_new)
```

To test your understanding, implement the **derivative()** function.

In [5]:
def derivative(a1, xi_list, yi_list):
    len_data = len(xi_list)
    error = 0
    for i in range(0, len_data):
        error += xi_list[i]*(a1*xi_list[i] - yi_list[i])
    deriv = 2*error/len_data
    return deriv

def gradient_descent(xi_list, yi_list, max_iterations, alpha, a1_initial):
    a1_list = [a1_initial]

    for i in range(0, max_iterations):
        a1 = a1_list[i]
        deriv = derivative(a1, xi_list, yi_list)
        a1_new = a1 - alpha*deriv
        a1_list.append(a1_new)
    return(a1_list)



In [6]:
param_iterations = gradient_descent(train['Gr Liv Area'], train['SalePrice'], 20, .0000003, 150)
print(param_iterations[-1])

120.142191472


4. Understanding Multi Parameter Gradient Descent
==

Now that we've understood how single parameter gradient descent works, let's build some intuition for multi parameter gradient descent. Let's start by visualizing the MSE as a function of the parameter values for the following simple linear regression model:

$$SalePrice=a_1∗GrLivArea+a_0$$

In this screen's widget, we've generated a 3D scatter plot with:

- a_0 on the x-axis
- a_1 on the y-axis
- MSE on the z-axis

In [7]:
import plotly 
plotly.tools.set_credentials_file(username='ivanovitch.slv', api_key='jFOZRqpKksZZcZqvrkkj')

In [8]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

# a_0
x = np.linspace(-100000,100000,1001)
# a_1
y = np.linspace(-400,600,1001)

# MSE
z = np.zeros(1001)
for i in range(0,1001):
    z[i] = sum((1/train['Gr Liv Area'].shape[0]) * ( (y[i]*train['Gr Liv Area'] + x[i]) 
                                                    - train['SalePrice'])**2)

trace1 = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=12,
            color='rgb(100,5,98)',              
            opacity=0.8
        )
)
    
data = [trace1]
layout = go.Layout(
    scene = dict(
                    xaxis = dict(
                        title='A0'),
                    yaxis = dict(
                        title='A1'),
                    zaxis = dict(
                        title='MSE'),
            ),
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='3d-scatter-colorscale')

5. Gradient Of The Cost Function
==

The gradient is a multi variable generalization of the derivative. In the last few screens, we were concerned with minimizing the following cost function:

$$MSE(a_1)=\frac{1}{n}\sum_{i=1}^{n}(a_1x_1^{(i)}−y^{(i)})^2$$

When we have 2 parameter values ($a_0$ and $a_1$), the cost function is now a function of 2 variables, not 1:

$$MSE(a_0,a_1)=\frac{1}{n}\sum_{i=1}^{n}(a_0+a_1x_1^{(i)}−y^{(i)})^2$$


Instead of one update rule, we now need two update rules. We need one for $a_0$:

$$a_0:=a_0−\alpha \frac{d}{da_0}MSE(a_0,a_1)$$

and one for $a_1$:

$$a_1:=a_1−\alpha \frac{d}{da_1}MSE(a_0,a_1)$$

Earlier in this mission, we determined that $\frac{d}{da_1}MSE(a_1)$ worked out to $\frac{2}{n}\sum_{i=1}^{n}x_1^{(i)}(a_1x_1^{(i)}−y^{(i)})$. For the multiparameter case, we need to include the additional parameter :

$$\frac{d}{da_1}MSE(a_0,a_1)=\frac{2}{n}\sum_{i=1}^{n}x_1^{(i)}(a_0+a_1x_1^{(i)}−y^{(i)})$$

For $\frac{d}{da_0}MSE(a_0,a_1)$, we won't walk through the proof for this derivative, but it's similar to the one we did for $a_1$ and **we encourage you to derive this yourself on pencil and paper**:

$$\frac{d}{da_0}MSE(a_0,a_1)=\frac{2}{n}\sum_{i=1}^{n}(a_0+a_1x_1^{(i)}−y^{(i)})$$


In [10]:
def a1_derivative(a0, a1, xi_list, yi_list):
    len_data = len(xi_list)
    error = 0
    for i in range(0, len_data):
        error += xi_list[i]*(a0 + a1*xi_list[i] - yi_list[i])
    deriv = 2*error/len_data
    return deriv

def a0_derivative(a0, a1, xi_list, yi_list):
    len_data = len(xi_list)
    error = 0
    for i in range(0, len_data):
        error += a0 + a1*xi_list[i] - yi_list[i]
    deriv = 2*error/len_data
    return deriv

def gradient_descent(xi_list, yi_list, max_iterations, alpha, a1_initial, a0_initial):
    a1_list = [a1_initial]
    a0_list = [a0_initial]

    for i in range(0, max_iterations):
        a1 = a1_list[i]
        a0 = a0_list[i]
        
        a1_deriv = a1_derivative(a0, a1, xi_list, yi_list)
        a0_deriv = a0_derivative(a0, a1, xi_list, yi_list)
        
        a1_new = a1 - alpha*a1_deriv
        a0_new = a0 - alpha*a0_deriv
        
        a1_list.append(a1_new)
        a0_list.append(a0_new)
    return(a0_list, a1_list)

a0_params, a1_params = gradient_descent(train['Gr Liv Area'], 
                                        train['SalePrice'], 20, .0000003, 150, 1000)
                                        
print(a0_params[-1],a1_params[-1])                                     

999.986114053 119.531794624


6. Gradient Descent For Higher Dimensions
==

What if we want to use many parameters in our model? Gradient descent actually scales to as many variables as you want. Each parameter value will need its own update rule, and it closely matches the update rule for $a_1$:

$$a_0:=a_0−\alpha \frac{d}{da_0}MSE$$
$$a_1:=a_1−\alpha \frac{d}{da_1}MSE$$
$$a_2:=a_2−\alpha \frac{d}{da_2}MSE$$
$$a_3:=a_3−\alpha \frac{d}{da_3}MSE$$
$$a_n:=a_n−\alpha \frac{d}{da_n}MSE$$

Besides the derivative for the MSE with respect to the intercept value ($a_0$), the derivative for other parameters are identical:

$$\frac{d}{da_1}MSE=\frac{2}{n}\sum_{i=1}^{n}x_1^{(i)}(\hat{y}^{(i)}−y^{(i)})$$
$$\frac{d}{da_2}MSE=\frac{2}{n}\sum_{i=1}^{n}x_2^{(i)}(\hat{y}^{(i)}−y^{(i)})$$
$$\frac{d}{da_n}MSE=\frac{2}{n}\sum_{i=1}^{n}x_n^{(i)}(\hat{y}^{(i)}−y^{(i)})$$


7. Next steps
==

In this mission, we explored how to find a linear regression model using the gradient descent algorithm. The main challenges with gradient descent include:

- choosing good initial parameter values
- choosing a good learning rate (falls under the domain of hyperparameter optimization)

In the next mission, we'll explore how to clean some of the remaining features in the training set to use in our model.